In [112]:
#!pip install -q imdb-sqlite
#!pip install -q pycountry

In [113]:
# imports
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style = "whitegrid")

In [114]:
%%time
!imdb-sqlite

Wall time: 259 ms


2022-04-19 22:22:35,622 DB already exists: (imdb.db). Refusing to modify. Exiting


Carregamento dos Dados

In [115]:
connection = sqlite3.connect("imdb.db")

In [116]:
tables = pd.read_sql_query("select name as 'Table_Name' from sqlite_master where type = 'table'", connection)
print("tables:", tables)
type(tables)

tables:   Table_Name
0     people
1     titles
2       akas
3       crew
4   episodes
5    ratings


pandas.core.frame.DataFrame

In [117]:
tables.head()

,Table_Name
0,people
1,titles
2,akas
3,crew
4,episodes


In [118]:
tables = tables["Table_Name"].values.tolist()
print("tables:", tables)

tables: ['people', 'titles', 'akas', 'crew', 'episodes', 'ratings']


In [119]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    result = pd.read_sql_query(query, connection)
    print("Esquema da tabela:", table)
    display(result)
    print("-"*100)
    print("\n")

Esquema da tabela: people


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,VARCHAR,0,None,1
1,1,name,VARCHAR,0,None,0
2,2,born,INTEGER,0,None,0
3,3,died,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: titles


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,type,VARCHAR,0,None,0
2,2,primary_title,VARCHAR,0,None,0
3,3,original_title,VARCHAR,0,None,0
4,4,is_adult,INTEGER,0,None,0
5,5,premiered,INTEGER,0,None,0
6,6,ended,INTEGER,0,None,0
7,7,runtime_minutes,INTEGER,0,None,0
8,8,genres,VARCHAR,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: akas


...

----------------------------------------------------------------------------------------------------


Esquema da tabela: episodes


,cid,name,type,notnull,dflt_value,pk
0,0,episode_title_id,INTEGER,0,None,0
1,1,show_title_id,INTEGER,0,None,0
2,2,season_number,INTEGER,0,None,0
3,3,eposide_number,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: ratings


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,rating,INTEGER,0,None,0
2,2,votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




1 - Quais são as categorias de filmes mais comuns no IMDB?

In [120]:
query1 = '''select type, count(*) as count from titles group by type order by 2 desc'''
result1 = pd.read_sql_query(query1, connection)
display("result1:", result1)

'result1:'

,type,count
0,tvEpisode,6651598
1,short,865592
2,movie,607710
3,video,259395
4,tvSeries,223734
5,tvMovie,135795
6,tvMiniSeries,43068
7,tvSpecial,36603
8,videoGame,30921
9,tvShort,10524


In [121]:
result1['percent'] = (result1['count'] / result1['count'].sum()) * 100
display(result1)

,type,count,percent
0,tvEpisode,6651598,75.032617
1,short,865592,9.764215
2,movie,607710,6.855206
3,video,259395,2.926077
4,tvSeries,223734,2.523807
5,tvMovie,135795,1.531821
6,tvMiniSeries,43068,0.485824
7,tvSpecial,36603,0.412896
8,videoGame,30921,0.348801
9,tvShort,10524,0.118715


In [122]:
others = {}
others['count'] = result1[result1['percent'] < 6]['count'].sum()
others['percent'] = result1[result1['percent'] < 6]['percent'].sum()
others['type'] = 'others'
print('others:', others)

others: {'count': 740042, 'percent': 8.347962118646686, 'type': 'others'}


In [123]:
result1 = result1[result1['percent'] >= 6]
result1 = result1.append(others, ignore_index=True)
result1 = result1.sort_values(by='percent', ascending=False)
display(result1)

,type,count,percent
0,tvEpisode,6651598,75.032617
1,short,865592,9.764215
3,others,740042,8.347962
2,movie,607710,6.855206
